In [1]:
from fynesse import access, assess, address

from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [2]:
from config import MODELS, EVENTS, SEEDS

## Data Clean-Up

In [3]:
for model in MODELS:
    for event in EVENTS:
        for seed in SEEDS:
            event_hash = address.hash(event)
            access.process(f'../experiments/results/core/{model}_{event_hash}_s{seed}.txt', f'./data/csv/core/{model}_{event_hash}_s{seed}.csv')

## Data Preprocessing

In [4]:
for seed in SEEDS:

    master_dict = dict()

    for model in MODELS:
        master_dict[model] = dict()
        curr_dict = master_dict[model]

        for event in EVENTS:
            event_hash = address.hash(event)

            df = access.load_df(f'./data/csv/core/{model}_{event_hash}_s{seed}.csv')
            inferences = assess.extract_inferences(df)

            curr_dict[event] = list(map(partial(address.make_timeseries, event=event), inferences))
            
    assess.make_pickle(f'./data/pickle/s{seed}.pickle', master_dict)

In [5]:
master_dict = dict()

for model in MODELS:
    master_dict[model] = dict()
    curr_dict = master_dict[model]

    for event in EVENTS:
        event_hash = address.hash(event)
        master_list = list()

        for seed in SEEDS:

            df = access.load_df(f'./data/csv/core/{model}_{event_hash}_s{seed}.csv')
            inferences = assess.extract_inferences(df)

            master_list += list(map(partial(address.make_timeseries, event=event), inferences))

        curr_dict[event] = master_list
            
assess.make_pickle(f'./data/pickle/master.pickle', master_dict)